In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
import random

import h5py
from IPython.utils import io

from joblib import Parallel, delayed
from tqdm import tqdm

In [2]:
%matplotlib widget

In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
import sys
sys.path.append('../../../../nadavp/new_trap/Tools/')
sys.path.append('../../../../nadavp/new_trap/Tools/StatFramework/')
import BeadDataFile
from discharge_tools import *

In [5]:
from likelihood_calculator import likelihood_analyser
from likelihood_calculator import GravityFramework
from likelihood_calculator import auxiliary_functions as aux
gfw = GravityFramework.GravityFramework()

In [7]:
bdf_discharge = load_dir(dirname='/data/new_trap/20210423/Bead3/Discharge/FineDischarge/', 
                         start_file=0, max_file=1000)
direction = np.array([bb.electrode_channels[0] for bb in bdf_discharge])
bdf_z_discharge = [bb for bb in bdf_discharge if bb.electrode_channels[0]==1]

150  files in folder
150  files loaded


In [9]:
bandwidth = 2
decimate = 10
charges = 4
m3 = gfw.build_z_response(bdf_z_discharge[80:100], drive_freq=71, charges=charges, bandwidth=bandwidth, decimate=decimate, bimodal=False)

***************************************************
Z2-amplitude:  2.39e+02
reduced chi2:  1544.931016864466
***************************************************
Z2-amplitude:  2.20e+02
reduced chi2:  1194.7697505973108
***************************************************
Z2-amplitude:  2.06e+02
reduced chi2:  1371.5516745466387
***************************************************
Z2-amplitude:  2.43e+02
reduced chi2:  1219.6911770979139
***************************************************
Z2-amplitude:  2.26e+02
reduced chi2:  1785.4344011285527
***************************************************
Z2-amplitude:  2.42e+02
reduced chi2:  1212.8175966488627
***************************************************
Z2-amplitude:  2.26e+02
reduced chi2:  1212.7127046418732
***************************************************
Z2-amplitude:  2.29e+02
reduced chi2:  1521.9010828334324
***************************************************
Z2-amplitude:  2.27e+02
reduced chi2:  612.4734862979749
************

In [10]:
scale = np.abs(np.mean([m_.values[0] for m_ in m3]))/charges
print("Scale is ", scale)

Scale is  58.03585767818004


In [30]:
bb = BeadDataFile.BeadDataFile(fname=r'/data/new_trap/20210423/Bead3/Discharge/Neutrality5/Discharge_0.h5')
bb.electrode_channels[0], np.std(bb.electrode_data[0])*np.sqrt(2)

(4, 0.020647085505769186)

In [42]:
ll = likelihood_analyser.LikelihoodAnalyser()
def func1_scale(i):
# for i,bdf_ in enumerate(data_bdfs2):
    print(i,'  ******************')
    fname = r'/data/new_trap/20210423/Bead3/Discharge/Neutrality4/Discharge_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)
    drive_freq = int(bb.cant_freq)
    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 3.15, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-1000, 1000], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    frequency = fit_kwargs['f']

    xx2 = bb.response_at_freq2('z', frequency, bandwidth=bandwidth)
    xx2 = xx2[5000:-5000:decimate]  # cut out the first and last second

    m1_tmp = ll.find_mle_sin(xx2, fsamp=500, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)


    return m1_tmp.values[0]

In [38]:
func1_scale(100)/scale

100   ******************


-0.29527409025202694

In [43]:
res_tmp = np.array(Parallel(n_jobs=40)(delayed(func1_scale)(i) for i in tqdm(range(999))))

100%|██████████| 999/999 [00:40<00:00, 24.90it/s]


In [40]:
_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(range(len(res_tmp)), res_tmp/scale, label='10V, 71Hz., NoSpin')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
_,ax = plt.subplots()
data = res_tmp/scale
_ = ax.hist(data, bins=30, range=(-0.5,0.5))
ax.set(xlabel=r'$\epsilon$')
np.mean(data), np.std(data)/np.sqrt(len(data))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-0.0034761108554813964, 0.005920536040966746)